In [140]:
from typing import Callable, Dict, Any, get_type_hints
from inspect import signature
from openai import Client
from dotenv import load_dotenv
import ast

In [141]:
load_dotenv()

True

In [142]:
def get_weather(loc:str,units:str)->str:
    if loc=='Hyd':
        return "25" +f" degrees {units}"
    else:
        return "50" +f" degrees {units}"

In [143]:
def add(a:int,b:int):
    return a+b

In [144]:
def say_hello():
    print("Hello")

In [267]:
class Tool:
    def __init__(self):
        self.tools:Dict = {}

    def register(self, func:Callable)->None:
        fn_signature = self.get_fn_signature(func)
        self.tools[fn_signature['name']] = func
        print("Tool Registered Successfully")
    
    def view_tools(self):
        print(self.tools)

    def get_fn_signature(self, fn: Callable) -> Dict[str, Any]:
        fn_signature = {
            "name": fn.__name__,
            "description": fn.__doc__ or "No description available.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }

        # Get parameter hints (handles missing annotations)
        type_hints = get_type_hints(fn)

        
        sig = signature(fn)

        for param_name, param in sig.parameters.items():
            if param_name == "return":
                continue  # Skip return type
            
            # Try to get the type, otherwise default to "any"
            param_type = type_hints.get(param_name, "any")
            param_type = param_type.__name__ if hasattr(param_type, "__name__") else "any"
            
            fn_signature["parameters"]["properties"][param_name] = {"type": param_type}
            fn_signature["parameters"]["required"].append(param_name)

        return fn_signature


    def tool_call(self, prompt):
        client = Client()
        
        function_definitions = "\n\n".join([
            f"Function Name: {fn_name}\nFunction Description: {func.__doc__ or 'No description available.'}\n"
            f"Function Signature: {self.get_fn_signature(func)['parameters']}"
            for fn_name, func in self.tools.items()
        ])

        
        system_message = f"""You are an helpful assistant who converts the user's query into function call if the user query needs the use of the function based on the given function definition so that the tool can be triggered and the context can be passed to LLM.\nBelow are the function definition:\n\n{function_definitions}
            """
        
        system_message += """\nThe output should be a Dictionary in the below format:\n{"name": "get_weather", "args": {"loc": "Hyd", "units":"C"}}"""
        
        response = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{'role':'system','content':system_message},
                        {'role':'user','content':f'For the given user query choose the best function and give me the function call as dict\nQuery: {prompt}'}]
        )
        
        return response.choices[0].message.content
    def call_function(self, prompt):
        function_call = self.tool_call(prompt)
        function_call = ast.literal_eval(function_call)
        function = self.tools[function_call['name']]
        kwargs = function_call['args']
        return function(**kwargs)


In [268]:
tools = Tool()
tools.register(get_weather)
tools.register(add)
tools.register(say_hello)


Tool Registered Successfully
Tool Registered Successfully
Tool Registered Successfully


In [282]:
class LLM:
    def __init__(self, Tool):
        self.tool = Tool
    
    def chat_with_tools(self, query):
        context = self.tool.call_function(query)
        print(context)
        client = Client()
        messages = [
            {'role':'system','content':f'The context below contains the answer to the users query. Use it to generate appropriate answer to the users query\n{context}'},
            {'role':'user','content':query}
        ]
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

In [286]:
llm = LLM(tools)
llm.chat_with_tools("What is the addition of 10 and -19")

-9


'The addition of 10 and -19 is -9.'